**notebook um iNaturalist 2019 Datensatz herunter zu laden, und nur tierbilder in googledrive zu speichern und diesen mit Megadetector bounding boxes zu versehen**

In [ ]:
#!wget https://ml-inat-competition-datasets.s3.amazonaws.com/2021/train_mini.tar.gz

In [ ]:
!git clone https://github.com/cocodataset/cocoapi

!cd /content/cocoapi/PythonAPI && make

In [ ]:
import os
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!wget https://ml-inat-competition-datasets.s3.amazonaws.com/2019/train2019.json.tar.gz

!wget https://ml-inat-competition-datasets.s3.amazonaws.com/2019/val2019.json.tar.gz

!wget https://ml-inat-competition-datasets.s3.amazonaws.com/2019/categories.json.tar.gz

In [ ]:
!tar -xvzf /content/categories.json.tar.gz
!tar -xvzf /content/train2019.json.tar.gz
!tar -xvzf /content/val2019.json.tar.gz

In [ ]:
import json

with open('/content/categories.json') as json_file:
    categories = json.load(json_file)

with open('/content/train2019.json') as json_file:
    training = json.load(json_file)

with open('/content/val2019.json') as json_file:
    validation = json.load(json_file)

In [ ]:
train_info = training['info']
val_info = validation['info']

train_images = training['images']
val_images = validation['images']

train_annotations = training['annotations']
val_annotations = validation['annotations']



In [ ]:
train_dataset = {'info': train_info, 'images':train_images, 'annotations':train_annotations, 'categories':categories}
val_dataset = {'info': val_info, 'images':val_images, 'annotations':val_annotations, 'categories':categories}

In [ ]:
with open('/content/train2019.json', 'w') as file:
  json.dump(train_dataset, file, indent = 4, sort_keys=False)

with open('/content/val2019.json', 'w') as file:
  json.dump(val_dataset, file, indent = 4, sort_keys=False)

In [ ]:
import json
from pycocotools.coco import COCO

train_file = '/content/train2019.json'
val_file = '/content/val2019.json'

train_coco = COCO(train_file)
val_coco = COCO(val_file)

In [ ]:
train_ids = train_coco.getImgIds()
val_ids = val_coco.getImgIds()

In [ ]:
delete_files = []

counter = 0

for ids in train_ids:
  #print(counter)
  counter = counter + 1
  ann_id = train_coco.getAnnIds([ids])
  ann = train_coco.loadAnns(ann_id)
  cat = ann[0]['category_id']
  if train_coco.loadCats([cat])[0]['class'] not in ['Aves', 'Reptilia', 'Amphibia']: #sollen auch amphibien drinnen sein?
    img = train_coco.loadImgs([ids])
    path = img[0]['file_name']
    delete_files.append(path)


entfernen aller annotationen+bilder die keine tiere enthalten

In [ ]:
for ids in val_ids:
  counter = counter + 1
  ann_id = val_coco.getAnnIds([ids])
  ann = val_coco.loadAnns(ann_id)
  cat = ann[0]['category_id']
  if val_coco.loadCats([cat])[0]['class'] not in ['Aves', 'Reptilia', 'Amphibia']: #sollen auch 
    img = val_coco.loadImgs([ids])
    path = img[0]['file_name']
    delete_files.append(path)

In [ ]:
with open('delete.json', 'w') as file:
  json.dump(delete_files, file, indent = 4, sort_keys=False)  #speichern einer liste aller bilder die entfernt werden sollen

In [ ]:
!tar -xvf /content/gdrive/MyDrive/Datasets/iNaturalist2019/zip/train_val2019.tar.gz --directory /content

In [ ]:
path_to_dataset = '/content/' 

for entry in delete_files:
  delete_path = path_to_dataset + entry
  os.remove(delete_path)
  #shutil.rmtree(delete_path)
  #print(delete_path)

In [ ]:
!rm -r /content/train_val2019/Plants

In [ ]:
!tar -cfvz /content/gdrive/MyDrive/Datasets/iNaturalist2019/zip/train_val2019.tar.gz /content/train_val2019

tar: Removing leading `/' from member names
^C


In [ ]:
cp -R /content/train_val2019 /content/gdrive/MyDrive/Datasets/iNaturalist2019/images

In [ ]:
drive.flush_and_unmount()

megadetector iNaturalist Datensatz erstellen und annotationsdatei speichern

In [ ]:
train_path = '/content/gdrive/MyDrive/Datasets/iNaturalist2019/annotations/train2019.json'
test_path = '/content/gdrive/MyDrive/Datasets/iNaturalist2019/annotations/val2019.json'
megadetector_path = '/content/gdrive/MyDrive/Datasets/iNaturalist2019/other/megadetector/output_mgdetector.json'

In [ ]:
import json
from pycocotools.coco import COCO

train = COCO(train_path)
test = COCO(test_path)

with open(megadetector_path) as json_file:
    megadetector = json.load(json_file)

meg_img = megadetector['images']
#print(len(meg_img))

In [ ]:
trn_ids = train.getImgIds()
tst_ids = test.getImgIds()

In [ ]:
images = []
annotations = []
categories = train.loadCats(train.getCatIds())

threshold = 0.5

counter = 0
failure = 0

for id in tst_ids:
  counter = counter + 1

  ann_id = test.getAnnIds([id])
  ann = test.loadAnns(ann_id)[0]

  cat = test.loadCats([ann['category_id']])

  if cat[0]['class'] in ['Amphibia', 'Reptilia', 'Aves']:
    img = test.loadImgs([id])[0]
    height = img['height']
    width = img['width']

    path = img['file_name']

    bbox_ann = [ann for ann in meg_img if ann['file'].split('/images/')[1] == path][0]

    if 'failure' not in bbox_ann.keys():

      if len(bbox_ann['detections']) > 0:

        detection = bbox_ann['detections'][0]

        conf = detection['conf']
        ct = detection['category']

        if ct == '1' and conf >= threshold:

          images.append(img)
          
          bbox = detection['bbox']
          ann['bbox'] = [bbox[0]*width, bbox[1]*height, bbox[2]*width, bbox[3]*height]

          annotations.append(ann)

    else:
      failure = failure + 1


# code for val_set

In [ ]:
images = []
annotations = []
categories = train.loadCats(train.getCatIds())

threshold = 0.5

counter = 0
failure = 0

for id in tst_ids:
  counter = counter + 1

  ann_id = test.getAnnIds([id])
  ann = test.loadAnns(ann_id)[0]

  cat = test.loadCats([ann['category_id']])

  if cat[0]['class'] in ['Amphibia', 'Reptilia', 'Aves']:
    img = test.loadImgs([id])[0]
    height = img['height']
    width = img['width']

    path = img['file_name']

    bbox_ann = [ann for ann in meg_img if ann['file'].split('/images/')[1] == path][0]

    images.append(img)
    bbox = [0,0,1,1]

    if 'failure' not in bbox_ann.keys():

      if len(bbox_ann['detections']) > 0:

        detection = bbox_ann['detections'][0]

        conf = detection['conf']
        ct = detection['category']

        if ct == '1' and conf >= threshold:
          
          bbox = detection['bbox']
    else:
      failure = failure + 1

    
    ann['bbox'] = [bbox[0]*width, bbox[1]*height, bbox[2]*width, bbox[3]*height]
    annotations.append(ann)

print(failure)

In [ ]:
categories = [cat for cat in categories if cat['class'] in ['Amphibia', 'Reptilia', 'Aves']] #bestimmen der gewollten kategorien

In [ ]:
with open('/content/gdrive/MyDrive/Datasets/iNaturalist2019/annotations/all_meg_test.json', 'w') as file: #speichern der erstellten annotationen in json file
  json.dump({'images':images, 'annotations':annotations, 'categories':categories}, file, indent = 4, sort_keys=False)

In [ ]:
drive.flush_and_unmount()